In [128]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

from pymongo import MongoClient
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Phrases
from gensim.corpora import Dictionary

from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [129]:
client = MongoClient()

In [130]:
client.paper.cs_paper_abs.find_one()['summary']

"Graph Neural Networks (GNNs) are limited in their expressive power, struggle\nwith long-range interactions and lack a principled way to model higher-order\nstructures. These problems can be attributed to the strong coupling between the\ncomputational graph and the input graph structure. The recently proposed\nMessage Passing Simplicial Networks naturally decouple these elements by\nperforming message passing on the clique complex of the graph. Nevertheless,\nthese models are severely constrained by the rigid combinatorial structure of\nSimplicial Complexes (SCs). In this work, we extend recent theoretical results\non SCs to regular Cell Complexes, topological objects that flexibly subsume SCs\nand graphs. We show that this generalisation provides a powerful set of graph\n``lifting'' transformations, each leading to a unique hierarchical message\npassing procedure. The resulting methods, which we collectively call CW\nNetworks (CWNs), are strictly more powerful than the WL test and, in

In [131]:
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
doc = tokenizer.tokenize(client.paper.cs_paper_abs.find_one()['summary'].lower())
doc = [lemmatizer.lemmatize(token) for token in doc]
print(doc)

['graph', 'neural', 'network', 'gnns', 'are', 'limited', 'in', 'their', 'expressive', 'power', 'struggle', 'with', 'long', 'range', 'interaction', 'and', 'lack', 'a', 'principled', 'way', 'to', 'model', 'higher', 'order', 'structure', 'these', 'problem', 'can', 'be', 'attributed', 'to', 'the', 'strong', 'coupling', 'between', 'the', 'computational', 'graph', 'and', 'the', 'input', 'graph', 'structure', 'the', 'recently', 'proposed', 'message', 'passing', 'simplicial', 'network', 'naturally', 'decouple', 'these', 'element', 'by', 'performing', 'message', 'passing', 'on', 'the', 'clique', 'complex', 'of', 'the', 'graph', 'nevertheless', 'these', 'model', 'are', 'severely', 'constrained', 'by', 'the', 'rigid', 'combinatorial', 'structure', 'of', 'simplicial', 'complex', 'sc', 'in', 'this', 'work', 'we', 'extend', 'recent', 'theoretical', 'result', 'on', 'sc', 'to', 'regular', 'cell', 'complex', 'topological', 'object', 'that', 'flexibly', 'subsume', 'sc', 'and', 'graph', 'we', 'show', 'th

In [146]:
class Corpus:
    def __init__(self, collection):
        self.collection = collection
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.lemmatizer = WordNetLemmatizer()

    def __iter__(self):
        docs = self.collection.find()
        for i, doc in enumerate(docs):
            doc = doc['summary'].lower().replace('_', '')
            doc = self.tokenizer.tokenize(doc)         
            doc = [token for token in doc if not token.isnumeric()]
            doc = [token for token in doc if len(token) > 1]
            doc = [self.lemmatizer.lemmatize(token) for token in doc]
            yield doc

class BigramCorpus:
    def __init__(self, data_gen, bigram):
        self.data_gen = data_gen
        self.bigram = bigram
    
    def __iter__(self):
        for i, doc in enumerate(self.data_gen):
            for token in self.bigram[doc]:
                if '_' in token:
                    doc.append(token)
            yield doc
            
class BOW:
    def __init__(self, data_gen, dictionary):
        self.data_gen = data_gen
        self.dictionary = dictionary
        
    def __iter__(self):
        for doc in self.data_gen:
            yield self.dictionary.doc2bow(doc)
            
def result_write(collection, bigram, dictionary, model):
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()
        
    docs = collection.find()
    result = {}
    for doc in docs:
        rawid = doc['_id']
        doc = doc['summary'].lower().replace('_', '')
        doc = tokenizer.tokenize(doc)         
        doc = [token for token in doc if not token.isnumeric()]
        doc = [token for token in doc if len(token) > 1]
        doc = [lemmatizer.lemmatize(token) for token in doc]
        for token in bigram[doc]:
            doc.append(token)
            
        x = dictionary.doc2bow(doc)
        scores = model[x]
        result[rawid] = sorted(scores, key=lambda ele:ele[1], reverse=True)[0][0]

    for rawid in result:
        collection.find_one_and_update({'_id': rawid}, {'$set': {'topic_index': result[rawid]}})
        

In [133]:
corpus = Corpus(client.paper.cs_paper_abs)
bigram = Phrases(corpus, min_count=20)
bigram_corpus = BigramCorpus(corpus, bigram)
dictionary = Dictionary(bigram_corpus)
dictionary.filter_extremes(no_below=20, no_above=0.5)

my_corpus = BOW(bigram_corpus, dictionary)

2021-06-25 11:15:53,391 : INFO : collecting all words and their counts
2021-06-25 11:15:53,395 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-06-25 11:15:59,016 : INFO : PROGRESS: at sentence #10000, processed 1690308 words and 531030 word types
2021-06-25 11:15:59,222 : INFO : collected 545731 token types (unigram + bigrams) from a corpus of 1752380 words and 10363 sentences
2021-06-25 11:15:59,223 : INFO : merged Phrases<545731 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
2021-06-25 11:15:59,224 : INFO : Phrases lifecycle event {'msg': 'built Phrases<545731 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000> in 5.83s', 'datetime': '2021-06-25T11:15:59.224032', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-74-generic-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-06-25 11:15:59,228 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-06-25 1

In [140]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 100
chunksize = 2000
passes = 20
iterations = 50
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=my_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2021-06-25 11:27:41,665 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
2021-06-25 11:27:41,667 : INFO : using serial LDA version on this node
2021-06-25 11:27:41,709 : WARNING : input corpus stream has no len(); counting documents
2021-06-25 11:27:49,245 : INFO : running online (multi-pass) LDA training, 100 topics, 20 passes over the supplied corpus of 10363 documents, updating model once every 2

2021-06-25 11:28:04,933 : INFO : topic #17 (0.009): 0.020*"malicious" + 0.020*"abstraction" + 0.019*"differentiable" + 0.018*"contact" + 0.017*"robustly" + 0.014*"triggered" + 0.012*"satisfaction" + 0.011*"minor" + 0.011*"application" + 0.010*"saddle"
2021-06-25 11:28:04,934 : INFO : topic #57 (0.012): 0.024*"network" + 0.017*"graph" + 0.016*"neural" + 0.015*"model" + 0.012*"can" + 0.009*"training" + 0.008*"performance" + 0.008*"method" + 0.008*"based" + 0.008*"layer"
2021-06-25 11:28:04,935 : INFO : topic #72 (0.012): 0.011*"method" + 0.009*"feature" + 0.008*"which" + 0.008*"model" + 0.007*"information" + 0.007*"from" + 0.007*"learning" + 0.007*"graph" + 0.006*"based" + 0.006*"show"
2021-06-25 11:28:04,935 : INFO : topic #28 (0.012): 0.027*"image" + 0.017*"method" + 0.011*"learning" + 0.010*"model" + 0.008*"from" + 0.007*"segmentation" + 0.007*"object" + 0.007*"feature" + 0.006*"training" + 0.006*"network"
2021-06-25 11:28:04,936 : INFO : topic diff=2.102772, rho=0.577350
2021-06-25 1

2021-06-25 11:28:17,460 : INFO : topic #23 (0.010): 0.064*"cloud" + 0.040*"monte" + 0.040*"monte_carlo" + 0.040*"carlo" + 0.026*"point" + 0.026*"apps" + 0.024*"carried_out" + 0.023*"map" + 0.023*"carried" + 0.023*"threat"
2021-06-25 11:28:17,461 : INFO : topic #62 (0.013): 0.025*"algorithm" + 0.016*"problem" + 0.010*"which" + 0.010*"guarantee" + 0.008*"constraint" + 0.007*"show" + 0.007*"least" + 0.007*"linear" + 0.007*"it" + 0.006*"matching"
2021-06-25 11:28:17,462 : INFO : topic #57 (0.013): 0.030*"network" + 0.025*"graph" + 0.020*"neural" + 0.017*"model" + 0.011*"can" + 0.010*"performance" + 0.009*"based" + 0.009*"gnns" + 0.008*"training" + 0.008*"method"
2021-06-25 11:28:17,462 : INFO : topic #28 (0.014): 0.038*"image" + 0.021*"method" + 0.010*"from" + 0.009*"feature" + 0.008*"model" + 0.008*"segmentation" + 0.008*"learning" + 0.007*"network" + 0.006*"face" + 0.006*"based"
2021-06-25 11:28:17,464 : INFO : topic diff=2.361434, rho=0.408248
2021-06-25 11:28:19,023 : INFO : PROGRESS: 

2021-06-25 11:28:32,632 : INFO : topic #62 (0.014): 0.028*"algorithm" + 0.018*"problem" + 0.013*"guarantee" + 0.011*"which" + 0.007*"show" + 0.007*"linear" + 0.007*"constraint" + 0.007*"it" + 0.007*"matching" + 0.006*"sampling"
2021-06-25 11:28:32,633 : INFO : topic #12 (0.015): 0.015*"system" + 0.009*"data" + 0.009*"software" + 0.009*"ha" + 0.009*"technology" + 0.009*"development" + 0.009*"paper" + 0.008*"application" + 0.008*"been" + 0.008*"science"
2021-06-25 11:28:32,634 : INFO : topic #28 (0.016): 0.046*"image" + 0.023*"method" + 0.010*"from" + 0.010*"feature" + 0.008*"segmentation" + 0.008*"model" + 0.007*"face" + 0.007*"propose" + 0.007*"we_propose" + 0.007*"network"
2021-06-25 11:28:32,635 : INFO : topic diff=2.315456, rho=0.373158
2021-06-25 11:28:34,067 : INFO : PROGRESS: pass 1, at document #8000/10363
2021-06-25 11:28:37,533 : INFO : optimized alpha [0.012852541, 0.013777148, 0.011592761, 0.012848001, 0.0117873, 0.0116461115, 0.012129236, 0.011324393, 0.010719623, 0.0119782

2021-06-25 11:28:43,432 : INFO : topic #12 (0.017): 0.015*"system" + 0.010*"technology" + 0.010*"research" + 0.010*"paper" + 0.009*"ha" + 0.008*"tool" + 0.008*"development" + 0.008*"been" + 0.008*"study" + 0.008*"their"
2021-06-25 11:28:43,433 : INFO : topic #28 (0.018): 0.048*"image" + 0.024*"method" + 0.011*"from" + 0.009*"feature" + 0.008*"segmentation" + 0.008*"face" + 0.008*"network" + 0.007*"based" + 0.007*"we_propose" + 0.007*"propose"
2021-06-25 11:28:43,434 : INFO : topic diff=2.114778, rho=0.373158
2021-06-25 11:28:44,862 : INFO : PROGRESS: pass 2, at document #2000/10363
2021-06-25 11:28:47,997 : INFO : optimized alpha [0.0147543615, 0.015888441, 0.0122589525, 0.013801983, 0.01274435, 0.012325994, 0.013373316, 0.01199623, 0.01089144, 0.01278927, 0.011711467, 0.013634726, 0.01836505, 0.012909119, 0.013244035, 0.012471706, 0.01266377, 0.011330688, 0.012955226, 0.013014015, 0.0131402975, 0.011525815, 0.013453819, 0.010888559, 0.012313768, 0.01287281, 0.010806063, 0.015046987, 0

...

2021-06-25 11:33:06,071 : INFO : topic #59 (0.027): 0.106*"privacy" + 0.044*"federated" + 0.031*"private" + 0.031*"fl" + 0.029*"data" + 0.027*"client" + 0.020*"preserving" + 0.020*"dp" + 0.018*"differential" + 0.018*"mechanism"
2021-06-25 11:33:06,071 : INFO : topic #49 (0.123): 0.167*"model" + 0.018*"generative" + 0.017*"prediction" + 0.015*"work" + 0.015*"generation" + 0.014*"bias" + 0.014*"metric" + 0.014*"evaluation" + 0.009*"such" + 0.008*"these"
2021-06-25 11:33:06,072 : INFO : topic #39 (0.180): 0.047*"been" + 0.035*"ha" + 0.033*"have" + 0.020*"research" + 0.019*"have_been" + 0.019*"ha_been" + 0.013*"these" + 0.012*"work" + 0.012*"there" + 0.012*"study"
2021-06-25 11:33:06,073 : INFO : topic #0 (0.336): 0.054*"be" + 0.046*"can" + 0.040*"not" + 0.025*"or" + 0.025*"can_be" + 0.024*"it" + 0.017*"but" + 0.015*"these" + 0.015*"they" + 0.014*"only"
2021-06-25 11:33:06,074 : INFO : topic diff=0.040047, rho=0.207697
2021-06-25 11:33:07,485 : INFO : PROGRESS: pass 17, at document #8000/1

2021-06-25 11:33:12,159 : INFO : topic #39 (0.181): 0.046*"been" + 0.034*"ha" + 0.033*"have" + 0.021*"research" + 0.020*"have_been" + 0.018*"ha_been" + 0.013*"these" + 0.012*"work" + 0.012*"study" + 0.011*"there"
2021-06-25 11:33:12,160 : INFO : topic #0 (0.344): 0.055*"be" + 0.047*"can" + 0.039*"not" + 0.026*"or" + 0.025*"can_be" + 0.024*"it" + 0.017*"but" + 0.015*"they" + 0.015*"these" + 0.014*"only"
2021-06-25 11:33:12,161 : INFO : topic diff=0.059895, rho=0.207697
2021-06-25 11:33:13,581 : INFO : PROGRESS: pass 18, at document #2000/10363
2021-06-25 11:33:14,839 : INFO : optimized alpha [0.35164577, 0.091971636, 0.051116895, 0.04301102, 0.0647468, 0.04639928, 0.06713885, 0.043143053, 0.025836509, 0.05607412, 0.03638915, 0.0484502, 0.09535162, 0.045684528, 0.042248785, 0.05157618, 0.034487043, 0.039766807, 0.05078152, 0.045794986, 0.049769383, 0.06998072, 0.046076614, 0.048952773, 0.03809849, 0.042278484, 0.029632937, 0.07645831, 0.079712175, 0.0628143, 0.0317988, 0.0583295, 0.04639

2021-06-25 11:33:20,538 : INFO : topic diff=0.037164, rho=0.203357
2021-06-25 11:33:21,957 : INFO : PROGRESS: pass 18, at document #8000/10363
2021-06-25 11:33:23,226 : INFO : optimized alpha [0.3864831, 0.09374953, 0.052608136, 0.043855265, 0.066236764, 0.047743544, 0.06816297, 0.04410895, 0.026488982, 0.057476472, 0.036944143, 0.049600303, 0.096109495, 0.046374276, 0.042666204, 0.052740943, 0.03499512, 0.040561806, 0.051877104, 0.04650244, 0.05066774, 0.071210794, 0.04680677, 0.049846325, 0.038596027, 0.04328018, 0.030267088, 0.07987091, 0.080696374, 0.064460345, 0.032240663, 0.059611566, 0.04734073, 0.039147478, 0.04730158, 0.04732567, 0.124909855, 0.074639395, 0.12964761, 0.18830967, 0.031036934, 0.059226673, 0.050846066, 0.04320906, 0.077785276, 0.08981567, 0.04627393, 0.07016396, 0.045222998, 0.128476, 0.05314656, 0.042619485, 0.046900757, 0.053157814, 0.112361476, 0.036750205, 0.07750636, 0.041171737, 0.07368036, 0.027448192, 0.044150315, 0.03816971, 0.10910667, 0.03478304, 0.07

2021-06-25 11:33:29,901 : INFO : merging changes from 2000 documents into a model of 10363 documents
2021-06-25 11:33:29,926 : INFO : topic #8 (0.027): 0.059*"automatic" + 0.054*"divergence" + 0.041*"chain" + 0.041*"non" + 0.037*"cross" + 0.036*"additive" + 0.034*"autoregressive" + 0.024*"error" + 0.023*"deletion" + 0.022*"between"
2021-06-25 11:33:29,927 : INFO : topic #59 (0.028): 0.107*"privacy" + 0.042*"federated" + 0.030*"private" + 0.030*"client" + 0.027*"fl" + 0.027*"data" + 0.020*"preserving" + 0.020*"differential" + 0.018*"mechanism" + 0.017*"differential_privacy"
2021-06-25 11:33:29,927 : INFO : topic #38 (0.129): 0.033*"model" + 0.031*"performance" + 0.027*"accuracy" + 0.023*"than" + 0.018*"can" + 0.017*"training" + 0.017*"while" + 0.014*"better" + 0.014*"achieve" + 0.014*"time"
2021-06-25 11:33:29,928 : INFO : topic #39 (0.190): 0.047*"been" + 0.036*"ha" + 0.033*"have" + 0.020*"research" + 0.020*"have_been" + 0.019*"ha_been" + 0.013*"these" + 0.013*"work" + 0.012*"study" + 

2021-06-25 11:33:38,583 : INFO : topic #59 (0.028): 0.109*"privacy" + 0.044*"federated" + 0.031*"private" + 0.030*"fl" + 0.029*"client" + 0.026*"data" + 0.022*"preserving" + 0.020*"dp" + 0.019*"mechanism" + 0.018*"differential"
2021-06-25 11:33:38,584 : INFO : topic #38 (0.135): 0.032*"model" + 0.032*"performance" + 0.026*"accuracy" + 0.023*"than" + 0.018*"can" + 0.017*"while" + 0.016*"training" + 0.014*"time" + 0.014*"state" + 0.014*"better"
2021-06-25 11:33:38,584 : INFO : topic #39 (0.195): 0.047*"been" + 0.037*"ha" + 0.034*"have" + 0.020*"research" + 0.020*"have_been" + 0.019*"ha_been" + 0.013*"these" + 0.013*"work" + 0.012*"study" + 0.012*"there"
2021-06-25 11:33:38,585 : INFO : topic #0 (0.431): 0.055*"be" + 0.048*"can" + 0.038*"not" + 0.026*"can_be" + 0.026*"or" + 0.025*"it" + 0.017*"but" + 0.015*"these" + 0.014*"which" + 0.014*"they"
2021-06-25 11:33:38,586 : INFO : topic diff=0.032199, rho=0.199278
2021-06-25 11:33:40,090 : INFO : PROGRESS: pass 19, at document #10000/10363
20

In [147]:
result_write(client.paper.cs_paper_abs, bigram, dictionary, model)

In [141]:
model.show_topics()

[(8,
  '0.058*"automatic" + 0.052*"divergence" + 0.046*"chain" + 0.039*"additive" + 0.039*"non" + 0.038*"cross" + 0.035*"autoregressive" + 0.024*"deletion" + 0.023*"error" + 0.023*"between"'),
 (59,
  '0.105*"privacy" + 0.039*"federated" + 0.029*"private" + 0.028*"client" + 0.028*"fl" + 0.027*"data" + 0.021*"differential" + 0.020*"dp" + 0.019*"differential_privacy" + 0.019*"shared"'),
 (98,
  '0.073*"learning" + 0.055*"rl" + 0.054*"reinforcement" + 0.051*"reward" + 0.042*"reinforcement_learning" + 0.041*"exploration" + 0.039*"algorithm" + 0.038*"environment" + 0.021*"based" + 0.021*"state"'),
 (26,
  '0.082*"phase" + 0.051*"consider" + 0.045*"taking" + 0.044*"we_consider" + 0.040*"post" + 0.036*"irs" + 0.034*"surface" + 0.025*"intelligent" + 0.022*"account" + 0.022*"aided"'),
 (40,
  '0.118*"protocol" + 0.051*"sigma" + 0.039*"party" + 0.038*"message" + 0.032*"blockchain" + 0.030*"transaction" + 0.030*"dialog" + 0.030*"customer" + 0.028*"secure" + 0.024*"communication"'),
 (36,
  '0.318

In [154]:
bigram.save('saved/bigram.bin')
dictionary.save('saved/dictionary.bin')
model.save('saved/ldamodel.bin')

2021-06-25 11:57:11,687 : INFO : Phrases lifecycle event {'fname_or_handle': 'saved/bigram.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-06-25T11:57:11.687050', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-74-generic-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2021-06-25 11:57:11,851 : INFO : saved saved/bigram.bin
2021-06-25 11:57:11,853 : INFO : Dictionary lifecycle event {'fname_or_handle': 'saved/dictionary.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-06-25T11:57:11.853768', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-74-generic-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2021-06-25 11:57:11,862 : INFO : saved saved/dictionary.bin
2021-06-25 11:57:11,865 : INFO : LdaState lifecycle event {'fname_or_handle': 'saved/ldamodel.bin.state', 'separate

In [142]:
text_iter = iter(corpus)
for doc in my_corpus:
    text = next(text_iter)
    scores = model[doc]
    cat = sorted(scores, key=lambda ele:ele[1], reverse=True)[0][0]
    if cat == 5:
        print(text)
        input('continue?')

['stereotactic', 'body', 'radiation', 'therapy', 'allows', 'for', 'precise', 'and', 'accurate', 'dose', 'delivery', 'organ', 'motion', 'during', 'treatment', 'bear', 'the', 'risk', 'of', 'undetected', 'high', 'dose', 'healthy', 'tissue', 'exposure', 'an', 'organ', 'very', 'susceptible', 'to', 'high', 'dose', 'is', 'the', 'oesophagus', 'it', 'low', 'contrast', 'on', 'ct', 'and', 'the', 'oblong', 'shape', 'render', 'motion', 'estimation', 'difficult', 'we', 'tackle', 'this', 'issue', 'by', 'modern', 'algorithm', 'to', 'measure', 'the', 'oesophageal', 'motion', 'voxel', 'wise', 'and', 'to', 'estimate', 'motion', 'related', 'dosimetric', 'impact', 'oesophageal', 'motion', 'wa', 'measured', 'using', 'deformable', 'image', 'registration', 'and', '4dct', 'of', 'internal', 'and', 'public', 'datasets', 'current', 'clinical', 'practice', 'of', 'contouring', 'the', 'organ', 'on', '3dct', 'wa', 'compared', 'to', 'timely', 'resolved', '4dct', 'contour', 'the', 'dosimetric', 'impact', 'of', 'the', '

KeyboardInterrupt: Interrupted by user

In [155]:
help(Dictionary.load)

Help on method load in module gensim.utils:

load(fname, mmap=None) method of abc.ABCMeta instance
    Load an object previously saved using :meth:`~gensim.utils.SaveLoad.save` from a file.
    
    Parameters
    ----------
    fname : str
        Path to file that contains needed object.
    mmap : str, optional
        Memory-map option.  If the object was saved with large arrays stored separately, you can load these arrays
        via mmap (shared memory) using `mmap='r'.
        If the file being loaded is compressed (either '.gz' or '.bz2'), then `mmap=None` **must be** set.
    
    See Also
    --------
    :meth:`~gensim.utils.SaveLoad.save`
        Save object to file.
    
    Returns
    -------
    object
        Object loaded from `fname`.
    
    Raises
    ------
    AttributeError
        When called on an object instance instead of class (this is a class method).

